In [1]:
## Import Library
import openpyxl
import collections
import os
from PIL import Image
import random
import json
from shutil import copyfile


import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm

import utils
import model.data_loader as data_loader
from evaluate import evaluate
import loss_and_metrics

import model.regression_adopted_cnn as regression_cnn
import model.data_loader as data_loader
from evaluate import evaluate
import regression_loss_and_metrics

h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date wei

# Recognizing Diseased Coffee Leaves Using Deep Learning


In this project, given a set of images of coffee leaves, this project will explore deep learning algorithms (both fully connected and convolution
neural networks) that output the correct labels for the conditions of the coffee leaves. 

The six conditions are 
* Healthy (H)
* Rust Level 1 (RL1)
* Rust Level 2 (RL2)
* Rust Level 3 (RL3)
* Rust Level 4 (RL4)
* Red Spider Mites (RSM)

For this project, we explore three different tasks:
1) Given the full dataset, classify them into the 6 categories mentioned above.
2) Given the full dataset, classify them into 3 categories (H, RL, RSM).
3) Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.

## Dataset
**Robusta dataset**: [Dataset](https://drive.google.com/drive/folders/13fFAQHU_-Ar0zg6RHl1FTLOE3I2QnCWI?usp=sharing)


### Setting the Virtual Environment and Installing Requirements
Requirements:
Run the follow commands:
```sh
$ pip install -r code/requirements.txt
```
Processing the Dataset
After downloading the annotations and images, they should be placed inside the CoffeeLeafNoteBook directory as follows

CoffeeLeafNoteBook/Annotations/{annotation files}
CoffeeLeafNoteBook/Photos/{.jpg files}

### To process the images for Task 1 above, run the following command:

* [Data Processing](#dataprocessing)

### To process the images for Task 2 above, run the following command:
* [Data Processing For Three Class](#dataprocessingthreeclass)

### To process the images for Task 3 above, run the following command:
* [Data Processing For RegressionTask](#dataprocessingregression)

### Training models
To train models, first create a ```params.json``` file inside the ```experiments/{A}/{B}``` directory, where
* {A} is either ```six_classes```, ```three_classes```, or ```regression```
* {B} is the descriptive name for the experiment model

**Then for Task 1 and 2, we use [Training the model](#trainmodel)**

**For Task 3, we use the [Train Regression](#trainmodelregression)**

### Evaluate on a Saved Model
**To evaluate on a saved model, run [Evaluate model](#evaluatef1)**


By default, it will evaluate on only the training and validation set.

+ To evaluate on the test set, the --testSet True flag must be added.
+ To not evaluate on the training and validation set, we can set the --trainAndVal False.
+ To evaluate only on the test set, we can set both flags --trainAndVal False --testSet True



## Data Processing <a class="anchor" id="dataprocessing"></a>
This is code for data processing task

**Given the full dataset, classify them into the 6 categories mentioned above.**

In [2]:

IMG_DIM = 720
xlsx_path = "./Annotations/RoCoLe-classes.xlsx"
annotation_json_path = "./Annotations/RoCoLe-json.json"
photo_path_prefix = "./Photos/"
binary_path = "./binary/"
multiclass_path = "./multiclass/"

binary_classifications = {
    "healthy": 0,
    "unhealthy": 1
}

multiclass_classifications = {
    "healthy": 0,
    "rust_level_1": 1,
    "rust_level_2": 2,
    "rust_level_3": 3,
    "rust_level_4": 4,
    "red_spider_mite": 5
}

def split_into_train_val_test(dict):
    random.seed(230)

    test = []
    val = []
    train = []

    for category in dict:
        img_names = list(dict[category])
        img_names.sort()
        random.shuffle(img_names)

        test_split = int(0.1 * len(img_names))
        val_split = int(.18 * len(img_names))

        test_img_names = img_names[:test_split]
        val_img_names = img_names[test_split: test_split + val_split]
        train_img_names = img_names[test_split + val_split:]

        test.extend(test_img_names)
        val.extend(val_img_names)
        train.extend(train_img_names)

    return {
        "test": set(test),
        "val": set(val),
        "train": set(train)
    }


def generate_binary_and_multiclass_dict_old():
    wb_obj = openpyxl.load_workbook(xlsx_path)
    sheet_obj = wb_obj.active

    binary_dict = collections.defaultdict(set)
    multiclass_dict = collections.defaultdict(set)

    num_row = sheet_obj.max_row

    for i in range(2, num_row + 1):
        image_name = sheet_obj.cell(row=i, column=1).value
        binary = sheet_obj.cell(row=i, column=2).value
        multiclass = sheet_obj.cell(row=i, column=3).value

        binary_dict[binary].add(image_name)
        multiclass_dict[multiclass].add(image_name)

    return binary_dict, multiclass_dict


def generate_train_val_test_split(binary_dict, multiclass_dict):
    binary_split = split_into_train_val_test(binary_dict)
    multiclass_split = split_into_train_val_test(multiclass_dict)
    return binary_split, multiclass_split


def get_split(img, split_dict):
    if img in split_dict["test"]:
        return "test"
    if img in split_dict["val"]:
        return "val"
    return "train"


def resize_and_save(filename, output_path, size=IMG_DIM):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(output_path)


def copy_photo_files_into_directories(classification_dict, new_classification_path, classification_type, split_dict):
    binary_or_multi = "binary" if "binary" in new_classification_path else "multiclass"
    for (category, images) in classification_dict.items():
        num = str(classification_type[category])
        for img in images:
            split = get_split(img, split_dict)
            make_dir(os.path.join("just_splitted", binary_or_multi, split))
            new_img_path = os.path.join("just_splitted", binary_or_multi, split, num + "_" + img)
            resize_and_save(os.path.join(photo_path_prefix, img), new_img_path)
            # copyfile(os.path.join("just_splitted", "cropped", img), new_img_path)


def categorize_train_val_test_split(verbose = False):
    (binary_dict, multiclass_dict) = generate_binary_and_multiclass_dict_old()
    if verbose:
        print("Finished categorizing pictures into their respective classes for binary and multiclass classification")
    binary_split, multiclass_split = generate_train_val_test_split(binary_dict, multiclass_dict)
    if verbose:
        print("Finished splitting dataset")
    # copy_photo_files_into_directories(binary_dict, binary_path, binary_classifications, binary_split)
    # if verbose:
    #     print("Finished copying photos into the 'binary' folder")
    copy_photo_files_into_directories(multiclass_dict, multiclass_path, multiclass_classifications, multiclass_split)
    if verbose:
        print("Finished copying photos into the 'multiclass' folder")


def make_dir(path):
    path = os.path.abspath(os.path.join(path))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory can't be made, because image cuts won't be saved.
            print('Error creating directory')
            raise e

def generate_binary_and_multiclass_dict(img_dimension = IMG_DIM):
    binary_dict = collections.defaultdict(set)
    multiclass_dict = collections.defaultdict(set)

    make_dir(os.path.join("zoom_cropped_and_splitted", "cropped"))
    with open(annotation_json_path) as json_file:
        data = json.load(json_file)
        ct = 0
        for pic_annotation in data:
            if ct % 25 == 0: print(ct)
            leaf_obj = pic_annotation["Label"]["Leaf"][0]
            geometry = leaf_obj["geometry"]
            img_name = pic_annotation["External ID"]

            binary_classif = leaf_obj["state"]
            multi_classif = pic_annotation["Label"]["classification"]
            classif_num = multiclass_classifications[multi_classif]

            image = Image.open(os.path.join(photo_path_prefix, img_name))
            width, height = image.size
            midx = int(width/2)
            midy = int(height/2)
            img_dimension_temp = img_dimension * 2
            zoom_cropped_img = image.crop((midx - img_dimension_temp, midy - img_dimension_temp, midx + img_dimension_temp, midy + img_dimension_temp))
            zoom_cropped_img_name = str(classif_num) + "_" + img_name
            zoom_cropped_img.save(os.path.join("zoom_cropped_and_splitted", "cropped", zoom_cropped_img_name))
            binary_dict[binary_classif].add(zoom_cropped_img_name)
            multiclass_dict[multi_classif].add(zoom_cropped_img_name)

            # for i in range(len(geometry)):
            #     xy = geometry[i]
            #
            #     x = xy["x"]
            #     y = xy["y"]
            #     xmin = x - img_dimension
            #     xmax = x + img_dimension
            #     ymin = y - img_dimension
            #     ymax = y + img_dimension
            #     if xmin < 0 or ymin < 0 or xmax > width or ymax > height:
            #         continue
            #
            #     new_img = image.crop((xmin, ymin, xmax, ymax))
            #     new_img_name = str(classif_num) + "_" + "{}_".format(i) + img_name
            #     new_img.save(os.path.join("cropped", new_img_name))
            #
            #     binary_dict[binary_classif].add(new_img_name)
            #     multiclass_dict[multi_classif].add(new_img_name)
            ct += 1

    return binary_dict, multiclass_dict

def main():
    categorize_train_val_test_split(True)


if __name__ == "__main__":
    main()

Finished categorizing pictures into their respective classes for binary and multiclass classification
Finished splitting dataset
Finished copying photos into the 'multiclass' folder


## Data Processing For Three Class <a class="anchor" id="dataprocessingthreeclass"></a>
This is code for data processing for three class

**Given the full dataset, classify them into 3 categories (H, RL, RSM).**

In [3]:
IMG_DIM = 720
xlsx_path = "./Annotations/RoCoLe-classes.xlsx"
photo_path_prefix = "./Photos/"


multiclass_classifications = {
    "healthy": 0,
    "rust_level_1": 1,
    "rust_level_2": 1,
    "rust_level_3": 1,
    "rust_level_4": 1,
    "red_spider_mite": 2
}

def split_into_train_val_test(dict):
    random.seed(230)

    test = []
    val = []
    train = []

    for category in dict:
        img_names = list(dict[category])
        img_names.sort()
        random.shuffle(img_names)

        test_split = int(0.1 * len(img_names))
        val_split = int(.18 * len(img_names))

        test_img_names = img_names[:test_split]
        val_img_names = img_names[test_split: test_split + val_split]
        train_img_names = img_names[test_split + val_split:]

        test.extend(test_img_names)
        val.extend(val_img_names)
        train.extend(train_img_names)

    return {
        "test": set(test),
        "val": set(val),
        "train": set(train)
    }


def generate_binary_and_multiclass_dict():
    wb_obj = openpyxl.load_workbook(xlsx_path)
    sheet_obj = wb_obj.active

    multiclass_dict = collections.defaultdict(set)

    num_row = sheet_obj.max_row

    for i in range(2, num_row + 1):
        image_name = sheet_obj.cell(row=i, column=1).value
        binary = sheet_obj.cell(row=i, column=2).value
        multiclass = sheet_obj.cell(row=i, column=3).value

        multiclass_dict[multiclass].add(image_name)

    return multiclass_dict


def generate_train_val_test_split(multiclass_dict):
    multiclass_split = split_into_train_val_test(multiclass_dict)
    return multiclass_split


def get_split(img, split_dict):
    if img in split_dict["test"]:
        return "test"
    if img in split_dict["val"]:
        return "val"
    return "train"


def resize_and_save(filename, output_path, size=IMG_DIM):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(output_path)


def copy_photo_files_into_directories(classification_dict, classification_type, split_dict):
    for (category, images) in classification_dict.items():
        num = str(classification_type[category])
        for img in images:
            split = get_split(img, split_dict)
            make_dir(os.path.join("three_classes", "multiclass", split))
            new_img_path = os.path.join("three_classes", "multiclass", split, num + "_" + img)
            resize_and_save(os.path.join(photo_path_prefix, img), new_img_path)


def categorize_train_val_test_split(verbose = False):
    multiclass_dict = generate_binary_and_multiclass_dict()
    if verbose:
        print("Finished categorizing pictures into their respective classes for multiclass classification")
    multiclass_split = generate_train_val_test_split(multiclass_dict)
    if verbose:
        print("Finished splitting dataset")
    copy_photo_files_into_directories(multiclass_dict, multiclass_classifications, multiclass_split)
    if verbose:
        print("Finished copying photos into the 'multiclass' folder")


def make_dir(path):
    path = os.path.abspath(os.path.join(path))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory can't be made, because image cuts won't be saved.
            print('Error creating directory')
            raise e

def main():
    categorize_train_val_test_split(True)


if __name__ == "__main__":
    main()

Finished categorizing pictures into their respective classes for multiclass classification
Finished splitting dataset
Finished copying photos into the 'multiclass' folder


## Data Processing For Regression Task <a class="anchor" id="dataprocessingregression"></a>
This is code for data processing for Regression Task

**Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.**


In [4]:
IMG_DIM = 720
xlsx_path = "./Annotations/RoCoLe-classes.xlsx"
photo_path_prefix = "./Photos/"


multiclass_classifications = {
    "healthy": 0,
    "rust_level_1": 1,
    "rust_level_2": 2,
    "rust_level_3": 3,
    "rust_level_4": 4,
    "red_spider_mite": 5
}

def split_into_train_val_test(dict):
    random.seed(230)

    test = []
    val = []
    train = []

    for category in dict:
        img_names = list(dict[category])
        img_names.sort()
        random.shuffle(img_names)

        test_split = int(0.1 * len(img_names))
        val_split = int(.18 * len(img_names))

        test_img_names = img_names[:test_split]
        val_img_names = img_names[test_split: test_split + val_split]
        train_img_names = img_names[test_split + val_split:]

        test.extend(test_img_names)
        val.extend(val_img_names)
        train.extend(train_img_names)

    return {
        "test": set(test),
        "val": set(val),
        "train": set(train)
    }


def generate_binary_and_multiclass_dict():
    wb_obj = openpyxl.load_workbook(xlsx_path)
    sheet_obj = wb_obj.active

    multiclass_dict = collections.defaultdict(set)

    num_row = sheet_obj.max_row

    for i in range(2, num_row + 1):
        image_name = sheet_obj.cell(row=i, column=1).value
        multiclass = sheet_obj.cell(row=i, column=3).value

        multiclass_dict[multiclass].add(image_name)

    return multiclass_dict


def generate_train_val_test_split(multiclass_dict):
    multiclass_split = split_into_train_val_test(multiclass_dict)
    return multiclass_split


def get_split(img, split_dict):
    if img in split_dict["test"]:
        return "test"
    if img in split_dict["val"]:
        return "val"
    return "train"


def resize_and_save(filename, output_path, size=IMG_DIM):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(filename)
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    image = image.resize((size, size), Image.BILINEAR)
    image.save(output_path)


def copy_photo_files_into_directories(classification_dict, classification_type, split_dict):
    for (category, images) in classification_dict.items():
        num = classification_type[category]
        if num == 5: continue
        num_str = str(num)
        for img in images:
            split = get_split(img, split_dict)
            make_dir(os.path.join("regression", "multiclass", split))
            new_img_path = os.path.join("regression", "multiclass", split, num_str + "_" + img)
            resize_and_save(os.path.join(photo_path_prefix, img), new_img_path)


def categorize_train_val_test_split(verbose = False):
    multiclass_dict = generate_binary_and_multiclass_dict()
    if verbose:
        print("Finished categorizing pictures into their respective classes for multiclass classification")
    multiclass_split = generate_train_val_test_split(multiclass_dict)
    if verbose:
        print("Finished splitting dataset")
    copy_photo_files_into_directories(multiclass_dict, multiclass_classifications, multiclass_split)
    if verbose:
        print("Finished copying photos into the 'multiclass' folder")


def make_dir(path):
    path = os.path.abspath(os.path.join(path))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory can't be made, because image cuts won't be saved.
            print('Error creating directory')
            raise e

def main():
    categorize_train_val_test_split(True)


if __name__ == "__main__":
    main()

Finished categorizing pictures into their respective classes for multiclass classification
Finished splitting dataset
Finished copying photos into the 'multiclass' folder


## Train Model For Task 1 and 2 <a class="anchor" id="trainmodel"></a>
This is code for training the model for task 1 and task 2

+ Task 1: Given the full dataset, classify them into the 6 categories mentioned above.
+ Task 2: Given the full dataset, classify them into 3 categories (H, RL, RSM).

In [6]:
# Choose model to train example mobilenet_baseline on six class
!python train.py --model_dir experiments/six_classes/mobilenet_baseline/

SyntaxError: invalid syntax (2906995662.py, line 2)

## Train Model For Task 3 <a class="anchor" id="trainmodelregression"></a>
This is code training the model for task 3 

**Task 3: Given the images from the healthy and rust level categories only, classify them into 5 categories (H, RL1, RL2, RL3, RL4) using a regression-based approach.**

In [7]:
!python train.py --model_dir experiments/regression/mobilenet_baseline

^C


## Evaluated Model <a class="anchor" id="evaluatef1"></a>
This is code evaluated model


In [9]:
# Can evaluate F1 score model after train example mobilenet_baseline on six classes
!python calculateF1Metrics.py --model_dir experiments/six_classes/mobilenet_baseline

^C


h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date wei

Unique Labels: 6 for 1128 total pictures


# Thử nghiệm dự đoán bệnh

In [7]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn

# Đường dẫn đến mô hình và ảnh cần kiểm tra
# Ở đây file model đó là đuôi .pth, ở đây đó là best.pth.tar ở experiments/six_classes/mobilenet_baseline/
model_path = "experiments/six_classes/mobilenet_baseline/best.pth.tar"
image_path = "H:/Coffee dataset/Coffee leaf in lab/coffee-leaf-diseases/test/images/64.jpg"

# Thiết lập mô hình
class MobileNet(nn.Module):
    def __init__(self, num_classes=6):
        super(MobileNet, self).__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)
        self.model.classifier[1] = nn.Linear(in_features=self.model.classifier[1].in_features, out_features=num_classes)
    
    def forward(self, x):
        return self.model(x)

# Tạo mô hình và load trọng số
model = MobileNet(num_classes=6)
checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# Load và xử lý ảnh
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

image = Image.open(image_path)
image = transform(image).unsqueeze(0)

# Dự đoán
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# In kết quả
labels = ['healthy', 'rust_level_1', 'rust_level_2', 'rust_level_3', 'rust_level_4', 'red_spider_mite']
print(f'This leaf is predicted to have: {labels[predicted.item()]}')


Using cache found in C:\Users\Viet/.cache\torch\hub\pytorch_vision_v0.10.0
h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
h:\AnacondaApp\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Viet\AppData\Local\Temp\ipykernel_4044\1219257838.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In

RuntimeError: Error(s) in loading state_dict for MobileNet:
	Missing key(s) in state_dict: "model.features.0.0.weight", "model.features.0.1.weight", "model.features.0.1.bias", "model.features.0.1.running_mean", "model.features.0.1.running_var", "model.features.1.conv.0.0.weight", "model.features.1.conv.0.1.weight", "model.features.1.conv.0.1.bias", "model.features.1.conv.0.1.running_mean", "model.features.1.conv.0.1.running_var", "model.features.1.conv.1.weight", "model.features.1.conv.2.weight", "model.features.1.conv.2.bias", "model.features.1.conv.2.running_mean", "model.features.1.conv.2.running_var", "model.features.2.conv.0.0.weight", "model.features.2.conv.0.1.weight", "model.features.2.conv.0.1.bias", "model.features.2.conv.0.1.running_mean", "model.features.2.conv.0.1.running_var", "model.features.2.conv.1.0.weight", "model.features.2.conv.1.1.weight", "model.features.2.conv.1.1.bias", "model.features.2.conv.1.1.running_mean", "model.features.2.conv.1.1.running_var", "model.fe